In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from tqdm import tqdm as ProgressBar
import six # needed for Google Cloud client
import operator
import sys
import en # NodeBox https://www.nodebox.net/code/index.php/Linguistics#verb_conjugation
from nltk.corpus import stopwords

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:
df=pd.read_pickle('CleanedIngredients.pkl')
cleanIngredients=pd.read_pickle('unique_ingredients.pkl')
print df.shape
print cleanIngredients[0:9]
print df[['instructionSteps']].values[0]
print df[['ingredients']].values[0]
print df[['cleanedIngredients']].values[0]


(89061, 11)
['', '(r) condensed cream celery soup', '(r) condensed cream mushroom soup', 'abalone shell', 'absinthe', 'acai berry pulp', 'acai berry sorbet', 'acai powder', 'acaipomegranate juice']
[ [u'In a bowl, mix the tomatoes, onion, red bell pepper, yellow bell pepper, cilantro, lime juice, and salt. Cover and refrigerate until ready to serve.']]
[ [u'6 roma (plum) tomatoes, diced', u'1 sweet onion, diced', u'1 medium red bell pepper, diced', u'1 medium yellow bell pepper, diced', u'1 bunch cilantro, finely minced', u'1 lime, juiced', u'1 teaspoon salt, or to taste', u'Add all ingredients to list', u'Add all ingredients to list']]
[ ['roma plum tomatoes', 'sweet onion', 'red bell pepper', 'yellow bell pepper', 'cilantro', 'lime', 'salt']]


In [3]:
def get_verbs(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    verbs=list()
    for token in tokens:
        if pos_tag[token.part_of_speech.tag] == 'VERB':
            verb=(token.text.content).lower()
            try:        
                verb=en.verb.present(verb)
            except:
                pass
            if verb not in stopWords:
                verbs.append(verb)
    time.sleep(.03)   # avoid Google API quota
    return(verbs)

In [4]:
for idx in range(10):
    instructions=df[['instructionSteps']].values[idx]
    verbs=list()
    for i in instructions[0]:
        verbs += get_verbs(i)
    print verbs
#print get_verbs(instruction)

['mix', 'refrigerate', 'serve']
['mix', 'chill', u'flavored', 'cut', 'arrange', 'sprinkle', 'desire', 'spray', 'repeat', 'remain', 'allow', 'cool', 'serve']
['add', 'remain', 'taste', 'melt', 'stir', 'thicken', 'continue', 'stir', 'melt', 'pour', 'cover']
[u'flavored', 'combine', 'stir', u'overbake', 'let', 'cool', 'remove', 'cool', 'make', u'get', 'last']
['mix', 'melt', 'turn', 'add', 'pour', 'simmer', 'turn', 'run']
['pour', 'cook', 'set', 'brush', 'cut', 'spread', 'serve', 'sprinkle']
['grease', 'stir', 'pan', 'mix', 'mix', 'divide', 'bake', 'insert', 'come']
['cook', 'stir', 'brown', 'mix', 'cover', 'bring', 'cover', 'reduce', 'simmer', 'absorb', 'stir', 'continue', 'simmer', u'heated', 'mix']
['heat', 'warm', 'combine', 'used', 'transfer', 'turn', 'coat', 'run', 'insert', 'read']
['grease', 'stir', 'melt', 'add', 'mix', 'combine', 'stir', 'fold', 'spread', 'bake', 'insert', 'come', 'remove', 'cool']


In [5]:
allVerbs = defaultdict(int)

# Load a list of stopwords
stopWords = set(stopwords.words('english'))

class BreakoutException(Exception): pass  # declare a label

try:
    for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
        if idx<22688:
            continue
        if  type(row['instructionSteps']) != float:
            instructions = '.'.join(instruction for instruction in row['instructionSteps'])
            try:
                verbs = get_verbs(instructions)
                for verb in verbs:
                    allVerbs[verb] += 1 
            except KeyboardInterrupt:
                raise BreakoutException
            except: 
                print idx 
                print instructions
                raise BreakoutException
except BreakoutException:
    pass
print 'Top 30 Verbs:'
for item in sorted(allVerbs.items(), reverse=True, key=operator.itemgetter(1))[:30]:
    print item

Processing recipes: 12474it [27:28,  6.78it/s]

TypeError: cannot concatenate 'str' and 'type' objects

In [8]:
len(allVerbs)

965

In [ ]:
verbVertor =  [get_verbs('.'.join(df[['instructionSteps']].values[i][0])) for i in range(df.shape[0])]
df['verbs']=verbVertor

In [12]:
# Store data (serialize to pickle file)
df.to_pickle('cleanedIngredients.pkl')
with open('verbs.pkl', 'wb') as handle:
    pickle.dump(allVerbs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df.sample(5)